In [1]:
from ortools.linear_solver import pywraplp
from datetime import datetime, date
import pandas as pd
import calendar

solver = pywraplp.Solver("Mixed Integer Programming", pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Variáveis do problema

In [2]:
month_idx = 11
now = datetime.now()
n_days = calendar.monthrange(now.year, month_idx)[1]
weekdays  = [date(now.year, month_idx, d).weekday() for d in range(1, n_days + 1)]
first_weekday = weekdays[0]
n_weeks = len(calendar.monthcalendar(now.year, month_idx))
print(n_weeks)

max_workload = 110

5


In [3]:
residents = [
    { "idx": 0, "name": "André", "type": "R1" },
    { "idx": 1, "name": "Maria", "type": "R1" },
    { "idx": 2, "name": "Cinthya", "type": "R1" },
    { "idx": 3, "name": "Rolando", "type": "R1" },
    { "idx": 4, "name": "Victor", "type": "R1" },
    { "idx": 5, "name": "Lucas", "type": "R2" },
    { "idx": 6, "name": "Giselia", "type": "R2" },
    { "idx": 7, "name": "Pedro", "type": "R2" },
    { "idx": 8, "name": "Bárbara", "type": "R2" },
    { "idx": 9, "name": "Douglas", "type": "R2" },
    { "idx": 10, "name": "Filipe", "type": "R3" },
]

n_res = len(residents)
res_types = list(set([r['type'] for r in residents]))
n_types = len(res_types)

In [4]:
enf_list = ['André', 'Maria', 'Cinthya', 'Rolando', 'Victor']
enf_list = [r for r in residents if r['name'] in enf_list]
enf_list

[{'idx': 0, 'name': 'André', 'type': 'R1'},
 {'idx': 1, 'name': 'Maria', 'type': 'R1'},
 {'idx': 2, 'name': 'Cinthya', 'type': 'R1'},
 {'idx': 3, 'name': 'Rolando', 'type': 'R1'},
 {'idx': 4, 'name': 'Victor', 'type': 'R1'}]

In [5]:
shifts = [
    { "idx": 0, "name": "HRC_dia", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 1, "name": "HRC_noite", "start_time": 19, "end_time": 7, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 2, "name": "ENF", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,1,1], "num_res": 2 },
    { "idx": 3, "name": "CC", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 2 },
    { "idx": 4, "name": "HRSam", "start_time": 19, "end_time": 24, "frequency": [1,1,1,0,0,0,0], "num_res": 2 },
    { "idx": 5, "name": "Amb", "start_time": 7, "end_time": 19, "frequency": [1,1,1,1,1,0,0], "num_res": 1 },
]

n_shifts = len(shifts)
shift_eq = ['CC']
shift_eq_list = [s for s in shifts if s['name'] in shift_eq]
shift_eq_list

[{'idx': 3,
  'name': 'CC',
  'start_time': 7,
  'end_time': 19,
  'frequency': [1, 1, 1, 1, 1, 0, 0],
  'num_res': 2}]

In [6]:
def convert_date(hour, day=1):
    return datetime.now().replace(day=day, hour=hour-1, minute=0, second=0, microsecond=0)

def get_time(shift):
    start = shift['start_time']
    end = shift['end_time']
    if end < start:
        return convert_date(start), convert_date(end, 2)
    return convert_date(start), convert_date(end)

def check_overlap(s1, s2):
    start1, end1 = get_time(s1)
    start2, end2 = get_time(s2)
    if end1 > start2 and end2 > start1:
        return 1

    return 0

overlaps = [[check_overlap(s1, s2) for s1 in shifts] for s2 in shifts]


In [7]:
for i, shift in enumerate(shifts):
    start, end = get_time(shift)
    shifts[i]['workload'] = (end - start).seconds//3600

## Variáveis de decisão

In [8]:
# Variável binária que guarda a escala do mês
# r: residente
# s: plantão
# d: dia do mês
Xrsd = [[[solver.BoolVar(f'X{r}{s}{d}') for d in range(n_days)] for s in range(n_shifts)] for r in range(n_res)]

# Variável que guarda a carga horária semanal de cada residente
# r: residente
# w: semana
Hrw = [[solver.NumVar(0, max_workload, f'H{r}{w}') for w in range(n_weeks)] for r in range(n_res)]

# Somatório da carga horária semanal de todos os residentes por semana para cada tipo de residente
# t: tipo de residente
# w: semana
Stw = [[solver.NumVar(0, solver.infinity(), f'S{t}{w}') for w in range(n_weeks)] for t in range(n_types)]

# Somatório da quantidade de plantões de cada tipo realizados por cada tipo de residente (R1, R2, R3)
# t: tipo de residente
# s: tipo de plantão
Pts = [[solver.NumVar(0, solver.infinity(), f'P{t}{s}') for s in range(n_shifts)] for t in range(n_types)]

# Somatório da quantidade de plantões de cada tipo realizados por cada  residente
# r: residente
# s: tipo de plantão
Rrs = [[solver.NumVar(0, solver.infinity(), f'R{r}{s}') for s in range(n_shifts)] for r in range(n_res)]

## Restrições

0. Ligando variáveis de decisão

In [9]:
# Carga horária semanal por residente
for r, resident in enumerate(residents):
    week_counter = 0
    for d, weekday in enumerate(weekdays):
        if weekday == first_weekday:
            ct = solver.Constraint(0, 0, f"workload - resident {resident['name']} - week {week_counter}")
            ct.SetCoefficient(Hrw[r][week_counter], -1)
            week_counter += 1

        for s, shift in enumerate(shifts):
            ct.SetCoefficient(Xrsd[r][s][d], shift['workload'])

In [10]:
# Somatório da carga horária semanal dos residentes por semana
for w in range(n_weeks):
    for t in range(n_types):
        res_list = [r for r in residents if r['type'] == res_types[t]]
        ct = solver.Constraint(0,0, f"Workload sum - week {w} - res_type {t}")
        ct.SetCoefficient(Stw[t][w], -1)
        for res in res_list:
            ct.SetCoefficient(Hrw[res['idx']][w], 1)

In [11]:
# Somatório da quantidade de plantões de cada tipo realizados por cada residente

for r, resident in enumerate(residents):
    for s, shift in enumerate(shifts):
        ct = solver.Constraint(0,0, f"Shift quantity - shift {shift['name']} - resident {resident['name']}")
        ct.SetCoefficient(Rrs[r][s], -1)
        for d in range(n_days):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

In [12]:
# Somatório da quantidade de plantões de cada tipo realizado por cada tipo de residente (R1, R2, R3)
for s in range(n_shifts):
    for t in range(n_types):
        ct = solver.Constraint(0, 0, f"Number of shifts - shift {s} - res_type {t}")
        ct.SetCoefficient(Pts[t][s], -1)
        res_list = [r for r in residents if r['type'] == res_types[t]]
        for res in res_list:
            ct.SetCoefficient(Rrs[res['idx']][s], 1)

1. Métrica de Justiça - Carga horária

In [13]:
precision = 0.1

for w in range(n_weeks):
    for t in range(n_types):
        res_list = [r for r in residents if r['type'] == res_types[t]]
        N = len(res_list)
        # Limites superiores
        # N*xi - (1+precision) Sum_x <=0 ... para cada i em N
        for res in res_list:
            ct = solver.Constraint(-solver.infinity(), 0, f"Upper limit - week {w} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Hrw[res['idx']][w], N)
            ct.SetCoefficient(Stw[t][w], -(1+precision))

        # Limites inferiores
        # N*xi - (1-precision) Sum_x >= 0 ... para cada i em N
        for i in range(0,N):
            ct = solver.Constraint(0, solver.infinity(), f"Lower limit - week {w} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Hrw[res['idx']][w], N)
            ct.SetCoefficient(Stw[t][w], -(1-precision))

2. Métrica de Justiça - Tipo de plantão

In [14]:
precision = 0.2

for shift in shift_eq_list:
    for t in range(n_types):
        res_list = [r for r in residents if r['type'] == res_types[t]]
        N = len(res_list)
        # Limites superiores
        # N*xi - (1+precision) Sum_x <=0 ... para cada i em N
        for res in res_list:
            ct = solver.Constraint(-solver.infinity(), 0, f"Upper limit - shift {shift['name']} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Pts[t][shift['idx']], -(1+precision))
            for d in range(n_days):
                ct.SetCoefficient(Xrsd[res['idx']][shift['idx']][d], N)

        # Limites inferiores
        # N*xi - (1-precision) Sum_x >= 0 ... para cada i em N
        for res in res_list:
            ct = solver.Constraint(0, solver.infinity(), f"Lower limit - shift {shift['name']} - res_type {t} - resident {res['name']}")
            ct.SetCoefficient(Pts[t][shift['idx']], -(1-precision))
            for d in range(n_days):
                ct.SetCoefficient(Xrsd[res['idx']][shift['idx']][d], N)

1. Garantir que tenha o número de residentes necessário em cada plantão

In [15]:
for s, shift in enumerate(shifts):
    for d, weekday in enumerate(weekdays):

        # Find the number of residents depending on weekday
        num_res = 0
        if shift['frequency'][weekday]:
            num_res = shift['num_res']

        ct = solver.Constraint(num_res, num_res, f"Day {d + 1} - {num_res} residents in {shift['name']}")
        soma = ""
        for r in range(n_res):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

2. Garantir que não haja choque de horário na escala

In [16]:
for d in range(n_days):
    for r, resident in enumerate(residents):
        for shift_idx, shift in enumerate(shifts):
            ct = solver.Constraint(0, 1, f"No time overlap in day {d + 1} for resident {resident['name']} - Shift {shift['name']}")
            for s, overlap in enumerate(overlaps[shift_idx]):
                ct.SetCoefficient(Xrsd[r][s][d], overlap)

3. Ambulatório só tem R1

In [17]:
s = 5
shift = shifts[s]
r_mais = [res for res in residents if res['type'] in ["R2", "R3"]]

for d in range(n_days):
    for resident in r_mais:
        r = resident['idx']
        ct = solver.Constraint(0, 0, f"No r+ in ambulatory - day {d + 1} - resident {resident['name']}")
        ct.SetCoefficient(Xrsd[r][s][d], 1)

4. HRC_dia/noite, HRSam, ENF e CC precisa de um R1 e um R2/R3 por turno

In [18]:
for d in range(n_days):
    for s, shift in enumerate(shifts[:-1]):
        ct = solver.Constraint(0, 0, f"R1+R2/R3 in day {d + 1} - shift {shift['name']}")
        for r, resident in enumerate(residents):
            is_r1 = 1 if resident['type'] == 'R1' else -1
            ct.SetCoefficient(Xrsd[r][s][d], is_r1)

5. R3 faz HRC_dia ou CC 1x por semana cada

In [19]:
r3_shifts = [shifts[i] for i in [0, 3]]
r3_list = [res for res in residents if res['type']  == "R3"]

for resident in r3_list:
    r = resident['idx']
    for shift in r3_shifts:
        s = shift['idx']
        week_counter = 0
        for d, weekday in enumerate(weekdays):
            if weekday == first_weekday:
                week_counter += 1
                ct = solver.Constraint(1, 1, f"R3 {resident['name']} working once in shift {shift['name']} - week {week_counter}")
            ct.SetCoefficient(Xrsd[r][s][d], 1)

6. R3 não faz parte dos demais plantões (HRC_noite, amb, enf, hrsam)

In [20]:
not_r3_shifts = [shifts[i] for i in [1, 2, 4, 5]]

ct = solver.Constraint(0, 0, f"R3 residents doesnt work in other shifts")
for resident in r3_list:
    r = resident['idx']
    for shift in not_r3_shifts:
        s = shift['idx']
        for d in range(n_days):
            ct.SetCoefficient(Xrsd[r][s][d], 1)

7. O R1 fica a semana toda na enfermaria

In [21]:
week_counter = 0
s = 2  # ENF

for d, weekday in enumerate(weekdays):
    if weekday == first_weekday:
        resident = enf_list[week_counter]
        r = resident['idx']
        week_counter += 1
    ct = solver.Constraint(1, 1, f"R1 {resident['name']} working on ENF - week {week_counter + 1}")
    ct.SetCoefficient(Xrsd[r][s][d], 1)

8. Máximo 24 horas seguidas de trabalho

In [22]:
# como implementar?

## Função objetivo

In [23]:
objective = solver.Objective()

for t in range(n_types):
    for w in range(n_weeks):
        objective.SetCoefficient(Stw[t][w], 1)

objective.SetMinimization()

In [24]:
solver.NumConstraints()

2817

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('OTIMA')

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    r = 10
    resident = residents[r]

    # for r, resident in enumerate(residents):
    print(resident['name'])

    for d in range(n_days):
        for s, shift in enumerate(shifts[:-1]):
            if round(Xrsd[r][s][d].solution_value(), 0):
                print(f"DIA {d + 1} - {shift['name']}")

Error: Kernel is dead

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    for r, resident in enumerate(residents):
        print(f"\nRESIDENTE {resident['name']}")
        for w in range(n_weeks):
            print(f"Semana {w + 1}: {round(Hrw[r][w].solution_value(), 0)}")

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    schedule = []

    for d, weekday in enumerate(weekdays):
        day_schedule = []
        for s, shift in enumerate(shifts):
            res_list = []
            for r, resident in enumerate(residents):
                if round(Xrsd[r][s][d].solution_value(), 0):
                    res_list.append(resident['name'])
            day_schedule.append(', '.join(res_list))
        schedule.append(day_schedule)

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    cols = [s['name'] for s in shifts]
    idx = [f'{d + 1}/{month_idx}' for d in range(n_days)]


    pd.DataFrame(schedule, columns=cols, index=idx)

In [ ]:
if status == pywraplp.Solver.OPTIMAL:
    for s, shift in enumerate(shifts):
        print(f"\n\nPLANTÃO {shift['name']}")
        for r, resident in enumerate(residents):
            soma = 0
            for d, weekday in enumerate(weekdays):
                soma += Xrsd[r][s][d].solution_value()
            print(f"{resident['name']}: {round(soma, 0)}")